In [18]:
# 1. Import library
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings

In [19]:
warnings.filterwarnings("ignore", message=".*load_learner.*insecure pickle.*")

# 2. Load the data
train_df = pd.read_csv('../Data/processed/0427_01/train_2025.csv') 
test_df = pd.read_csv('../Data/processed/0427_01/test_2025.csv') 

train_df.drop(columns="claim_number", inplace=True)
test_id = test_df['claim_number']
test_df.drop(columns=["claim_number"], inplace=True)

In [ ]:
predictor = TabularPredictor(
    label="fraud",
    eval_metric="f1",
    problem_type="binary",
    path="../AutogluonModels/"
).fit(
    train_data=train_df,
    holdout_frac=0.2,
    presets="best",
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels\ag-20250428_035510"
Preset alias specified: 'best' maps to 'best_quality'.
2025-04-27 23:55:11,370	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-27 23:55:13,990	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(_dystack pid=30196) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
(_dystack pid=30196) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
(_dystack pid=30196) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.l

In [16]:
predictor = TabularPredictor(
    label="fraud",
    eval_metric="f1",
    problem_type="binary",
    verbosity=2
)

predictor.fit(
    train_data=train_df,
    holdout_frac=0.2,
    hyperparameters={
        'GBM': {  # LightGBM
            'num_boost_round': 1000,
            'learning_rate': 0.01,
            'num_leaves': 32,
            'extra_trees': True,
            'ag_args_fit': {'early_stop': True},
        }
    },
    hyperparameter_tune_kwargs={
        'num_trials': 20,  # Try 20 different parameter sets
        'scheduler': 'local',  # Run locally
        'searcher': 'random',  # Simple random search
    }
)

No path specified. Models will be saved in: "AutogluonModels\ag-20250428_101724"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       17.14 GB / 31.93 GB (53.7%)
Disk Space Avail:   287.70 GB / 935.97 GB (30.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'       

In [13]:
temp = predictor.leaderboard()

In [15]:
temp.to_csv('temp.csv')

In [17]:
# 4. Predict on the test set
test_df = pd.read_csv('../Data/processed/0427_01/test_2025.csv')
predictions = predictor.predict(test_df)

# 5. Save predictions to CSV
submission = pd.DataFrame({
    "claim_number": test_id,  # Important: use the original claim_number
    "fraud": predictions                      # Your predicted fraud labels (0 or 1)
})
submission.to_csv("../Submit/submissions/submission_2.csv", index=False)